# 1. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import warnings
import gdown  # Библиотека для скачивания с Google Диска

# Настройки
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

print("✅ Библиотеки загружены.")

# --- СКАЧИВАНИЕ ДАННЫХ ---
# Вставь сюда ID файла, который ты скопировал из ссылки
file_id = '1Jggcj85IBQNaXqBtVGUL6dfjP_WnSGbD'

# Формируем прямую ссылку для скачивания
url = f'https://drive.google.com/uc?id={file_id}'
output_file = 'movies_data.csv' # Имя, под которым файл сохранится в Colab

# Скачиваем файл
print("⏳ Скачиваем файл с данными...")
gdown.download(url, output_file, quiet=False)
print("✅ Файл успешно скачан.")

✅ Библиотеки загружены.
⏳ Скачиваем файл с данными...


Downloading...
From: https://drive.google.com/uc?id=1Jggcj85IBQNaXqBtVGUL6dfjP_WnSGbD
To: /content/movies_data.csv
100%|██████████| 797k/797k [00:00<00:00, 10.2MB/s]

✅ Файл успешно скачан.


# 2. Загрузка данных

Чтение основного датасета фильмов.

**Процесс загрузки:**
* Чтение CSV файла по указанному пути.
* Вывод размерности (количество строк и столбцов).
* Вывод первых строк для визуальной проверки корректности данных.

In [ ]:
# 2. Чтение данных

try:
    df = pd.read_csv('movies_data.csv')
    print(f"\n✅ Файл открыт! Размер таблицы: {df.shape}")
    display(df.head(3))
except FileNotFoundError:
    print("❌ Ошибка: Файл не скачался. Проверьте ID файла в предыдущем блоке.")



✅ Файл открыт! Размер таблицы: (5562, 10)


,Movie_Title,Year,Director,Actors,Rating,Runtime(Mins),Censor,Total_Gross,main_genre,side_genre
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...",9.3,148,UA,Gross Unkown,Action,"Adventure, Drama"
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,152,UA,$534.86M,Action,"Crime, Drama"
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",9.0,201,U,$377.85M,Action,"Adventure, Drama"


# 3. Предобработка данных (Feature Engineering)

На данном этапе мы готовим данные для модели.

**Алгоритм действий:**
1.  **Проверка:** Сначала проверяем, есть ли пропуски в ключевых колонках (`main_genre`, `side_genre`, `Year`, `Rating`).
2.  **Очистка:**
    * Пропуски в жанрах заполняем маркером `'Unknown'`.
    * Из столбца `Year` извлекаем числовое значение года. Пропуски заполняем медианой.
    * Пропуски в `Rating` заполняем средним значением (нужно для сортировки).
3.  **Кодирование:** Преобразуем текстовые жанры в числа через `One-Hot Encoding`.
4.  **Масштабирование:** Нормализуем год выпуска, чтобы сбалансировать его вес относительно жанров.

> ⚠️ **Важно:** Мы формируем таблицу `X_final` только из **Жанров** и **Года**.

In [ ]:
# 1. ПРОВЕРКА ПРОПУСКОВ
cols_to_check = ['main_genre', 'side_genre', 'Year', 'Rating']
missing_values = df[cols_to_check].isnull().sum()

print("📊 Статистика пропусков ДО обработки:")
print(missing_values)
print("-" * 40)

# 2. ОБРАБОТКА (Заполнение пропусков)

# Жанры: Заполняем 'Unknown'
if missing_values['main_genre'] > 0 or missing_values['side_genre'] > 0:
    df['main_genre'] = df['main_genre'].fillna('Unknown')
    df['side_genre'] = df['side_genre'].fillna('Unknown')
    print("✅ Пропуски в жанрах заполнены.")
else:
    print("✨ Пропусков в жанрах нет.")

# Год: Очистка и заполнение
# Сначала превращаем строку в число (извлекаем 4 цифры)
df['Year_Clean'] = df['Year'].astype(str).str.extract(r'(\d{4})').astype(float)

# Проверяем, появились ли NaN после извлечения (или были ли они до этого)
if df['Year_Clean'].isnull().sum() > 0:
    median_year = df['Year_Clean'].median()
    df['Year_Clean'] = df['Year_Clean'].fillna(median_year)
    print(f"✅ Пропуски в годах заполнены медианой ({int(median_year)}).")

# Рейтинг: Заполняем средним (только для сортировки, в модель не пойдет)
if df['Rating'].isnull().sum() > 0:
    mean_rating = df['Rating'].mean()
    df['Rating'] = df['Rating'].fillna(mean_rating)
    print(f"✅ Пропуски в рейтинге заполнены средним ({mean_rating:.1f}).")

print("-" * 40)

# 3. ПОДГОТОВКА ТАБЛИЦЫ ДЛЯ МОДЕЛИ (X_final)

# Кодируем жанры (One-Hot Encoding)
X_genres = pd.get_dummies(df[['main_genre', 'side_genre']], prefix=['main', 'side'])

# Масштабируем Год (StandardScaler)
scaler = StandardScaler()
X_year = scaler.fit_transform(df[['Year_Clean']])
X_year = pd.DataFrame(X_year, columns=['Year_Scaled'], index=df.index)

# Объединяем: Жанры + Год
X_final = pd.concat([X_genres, X_year], axis=1)

print("✅ Финальная таблица признаков собрана.")
print(f"Размер X_final: {X_final.shape}")

📊 Статистика пропусков ДО обработки:
main_genre    0
side_genre    0
Year          0
Rating        0
dtype: int64
----------------------------------------
✨ Пропусков в жанрах нет.
----------------------------------------
✅ Финальная таблица признаков собрана.
Размер X_final: (5562, 158)


# 4. Обучение моделей (KNN)

Мы создадим и обучим **три отдельные модели** на основе алгоритма K-Nearest Neighbors (KNN), каждая из которых будет использовать свой способ измерения "похожести" (дистанции).

**Список моделей:**
1.  **Cosine (Косинусное расстояние):** Смотрит на угол между векторами. Идеально подходит для жанров.
2.  **Euclidean (Евклидово расстояние):** Классическое расстояние по прямой.
3.  **Manhattan (Манхэттенское расстояние):** Измеряет путь по "сетке", устойчиво к выбросам.

> **Настройка:** Мы ставим `n_neighbors=20`. Это значит, что каждая модель найдет 20 ближайших фильмов, из которых мы позже выберем 5 лучших по рейтингу.

In [ ]:
# Создаем словарь для хранения моделей
models = {}

print("Начинаем обучение моделей...")


# МОДЕЛЬ 1: KNN с Косинусным расстоянием (Cosine)

print("1. Обучаем модель с метрикой Cosine...")
model_cosine = NearestNeighbors(n_neighbors=20, metric='cosine', n_jobs=-1)
model_cosine.fit(X_final) # Обучаем на нашей таблице (Жанры + Год)
models['Cosine'] = model_cosine


# МОДЕЛЬ 2: KNN с Евклидовым расстоянием (Euclidean)

print("2. Обучаем модель с метрикой Euclidean...")
model_euclidean = NearestNeighbors(n_neighbors=20, metric='euclidean', n_jobs=-1)
model_euclidean.fit(X_final)
models['Euclidean'] = model_euclidean


# МОДЕЛЬ 3: KNN с Манхэттенским расстоянием (Manhattan)

# Расстояние "городских кварталов" (сумма разниц по модулю).
print("3. Обучаем модель с метрикой Manhattan...")
model_manhattan = NearestNeighbors(n_neighbors=20, metric='manhattan', n_jobs=-1)
model_manhattan.fit(X_final)
models['Manhattan'] = model_manhattan

print("\n🎉 Все три модели успешно обучены!")
print("Теперь мы можем сравнивать их рекомендации.")

Начинаем обучение моделей...
1. Обучаем модель с метрикой Cosine...
2. Обучаем модель с метрикой Euclidean...
3. Обучаем модель с метрикой Manhattan...

🎉 Все три модели успешно обучены!
Теперь мы можем сравнивать их рекомендации.


# 5. Контрольный тест моделей

Перед тем как делать интерактивный поиск, давайте проверим работоспособность всех трех моделей на конкретном примере.

Мы возьмем фильм по индексу **50** (Jones and the Last Crusade (1989)) и посмотрим, какие **Топ-10** рекомендаций выдаст каждая из метрик: `Cosine`, `Euclidean`, `Manhattan`.

**Что мы проверяем:**
1.  Работает ли сортировка по рейтингу.
2.  Отличаются ли выдачи разных моделей (они должны немного отличаться, так как математика разная).

In [ ]:
def test_all_models_by_index(movie_index):
    # Проверка на корректность индекса
    if movie_index < 0 or movie_index >= len(df):
        print(f"❌ Индекс {movie_index} вне диапазона (0 - {len(df)-1})")
        return

    # 1. Получаем данные исходного фильма
    original_movie = df.iloc[movie_index]
    print(f"🧪 ТЕСТОВЫЙ ЗАПУСК для фильма (index {movie_index}):")
    print(f"🎬 {original_movie['Movie_Title']} ({int(original_movie['Year_Clean'])})")
    print(f"   Жанры: {original_movie['main_genre']}, {original_movie['side_genre']}")
    print(f"   ★ Рейтинг: {original_movie['Rating']}")
    print("=" * 60)

    # 2. Проходим по всем обученным моделям
    for model_name, model in models.items():
        print(f"\n⚙️ МОДЕЛЬ: {model_name}")
        print("-" * 30)

        # Ищем соседей (20 штук)
        distances, indices = model.kneighbors([X_final.iloc[movie_index]])

        # Исключаем первый элемент (сам фильм)
        neighbor_indices = indices[0][1:]
        neighbor_distances = distances[0][1:]

        # Формируем список кандидатов
        candidates = df.iloc[neighbor_indices].copy()
        candidates['Distance'] = neighbor_distances

        # СОРТИРОВКА: Берем топ-10 с наивысшим рейтингом
        top_10 = candidates.sort_values(by='Rating', ascending=False).head(5)

        # Выводим результат
        for i, (idx, row) in enumerate(top_10.iterrows(), 1):
            print(f"{i}. {row['Movie_Title']} ({int(row['Year_Clean'])})")
            print(f"   Жанры: {row['main_genre']}, {row['side_genre']}")
            print(f"   ★ {row['Rating']} | dist: {row['Distance']:.4f}")

# --- ЗАПУСК ТЕСТА ---

test_index = 50
test_all_models_by_index(test_index)

🧪 ТЕСТОВЫЙ ЗАПУСК для фильма (index 50):
🎬 Indiana Jones and the Last Crusade (1989)
   Жанры: Action,  Adventure
   ★ Рейтинг: 8.2

⚙️ МОДЕЛЬ: Cosine
------------------------------
1. Star Wars (1977)
   Жанры: Action,  Adventure,  Fantasy
   ★ 8.6 | dist: 0.3291
2. Raiders of the Lost Ark (1981)
   Жанры: Action,  Adventure
   ★ 8.4 | dist: 0.0238
3. Sholay (1975)
   Жанры: Action,  Adventure,  Comedy
   ★ 8.1 | dist: 0.3288
4. The Taking of Pelham One Two Three (1974)
   Жанры: Action,  Crime,  Thriller
   ★ 7.6 | dist: 0.3289
5. Batman (1989)
   Жанры: Action,  Adventure
   ★ 7.5 | dist: 0.0000

⚙️ МОДЕЛЬ: Euclidean
------------------------------
1. Raiders of the Lost Ark (1981)
   Жанры: Action,  Adventure
   ★ 8.4 | dist: 0.4956
2. Indiana Jones and the Last Crusade (1989)
   Жанры: Action,  Adventure
   ★ 8.2 | dist: 0.0000
3. Dip huet seung hung (1989)
   Жанры: Action,  Crime,  Drama
   ★ 7.8 | dist: 1.4142
4. Indiana Jones and the Temple of Doom (1984)
   Жанры: Action,  Adv

# 6. Интерактивный поиск рекомендаций

Финальная часть проекта. Здесь мы объединяем все наработки в удобную функцию.

Как пользоваться:
1.  Запустите ячейку.
2.  Появится поле ввода. Напишите название фильма на английском (например, `Matrix`, `Batman`, `Shrek`).
3.  Система найдет фильм в базе, подберет 20 похожих по жанрам и году, а затем покажет Топ-10 с самым высоким рейтингом.

> 💡  По умолчанию используется модель `Cosine`, так как она показала наиболее разнообразные и смысловые результаты на тестах.

In [ ]:
def get_recommendations_all_models(movie_title):
    # 1. Поиск фильма в базе
    matches = df[df['Movie_Title'].str.contains(movie_title, case=False, na=False)]

    if len(matches) == 0:
        print(f"❌ Фильм '{movie_title}' не найден. Попробуйте другое название.")
        return

    # Берем первый найденный фильм
    idx = matches.index[0]
    original_movie = df.iloc[idx]

    print(f"🎬 Вы выбрали: {original_movie['Movie_Title']} ({int(original_movie['Year_Clean'])})")
    print(f"   Жанры: {original_movie['main_genre']}, {original_movie['side_genre']}")
    print(f"   ★ Рейтинг: {original_movie['Rating']}")
    print("=" * 60)

    # 2. Проходим по всем моделям (Cosine, Euclidean, Manhattan)
    for model_name, model in models.items():
        print(f"\n🧠 МОДЕЛЬ: {model_name}")
        print("-" * 40)

        # Ищем соседей
        distances, indices = model.kneighbors([X_final.iloc[idx]])

        # Исключаем сам фильм
        neighbor_indices = indices[0][1:]
        neighbor_distances = distances[0][1:]

        # Формируем список кандидатов
        candidates = df.iloc[neighbor_indices].copy()
        candidates['Distance'] = neighbor_distances

        # 3. СОРТИРОВКА: Берем Топ-5 лучших по рейтингу
        top_5 = candidates.sort_values(by='Rating', ascending=False).head(5)

        # Вывод
        for i, (id_row, row) in enumerate(top_5.iterrows(), 1):
            print(f"{i}. {row['Movie_Title']} ({int(row['Year_Clean'])})")
            print(f"   Жанры: {row['main_genre']}, {row['side_genre']}")
            print(f"   ★ {row['Rating']} | dist: {row['Distance']:.4f}")

# --- ЗАПУСК ---
print("👇 Введите название фильма на английском:")
user_input = input("Название фильма: ")

if user_input.strip():
    get_recommendations_all_models(user_input)
else:
    print("⚠️ Ввод пуст.")

👇 Введите название фильма на английском:
Название фильма: The matrix
🎬 Вы выбрали: The Matrix (1999)
   Жанры: Action,  Sci-Fi
   ★ Рейтинг: 8.7

🧠 МОДЕЛЬ: Cosine
----------------------------------------
1. Terminator 2: Judgment Day (1991)
   Жанры: Action,  Sci-Fi
   ★ 8.6 | dist: 0.0483
2. The Terminator (1984)
   Жанры: Action,  Sci-Fi
   ★ 8.1 | dist: 0.1342
3. The Matrix Reloaded (2003)
   Жанры: Action,  Sci-Fi
   ★ 7.2 | dist: 0.0151
4. The Wolverine (2013)
   Жанры: Action,  Sci-Fi
   ★ 6.7 | dist: 0.1663
5. The Matrix Revolutions (2003)
   Жанры: Action,  Sci-Fi
   ★ 6.7 | dist: 0.0151

🧠 МОДЕЛЬ: Euclidean
----------------------------------------
1. Terminator 2: Judgment Day (1991)
   Жанры: Action,  Sci-Fi
   ★ 8.6 | dist: 0.4956
2. The Terminator (1984)
   Жанры: Action,  Sci-Fi
   ★ 8.1 | dist: 0.9292
3. The Boondock Saints (1999)
   Жанры: Action,  Thriller
   ★ 7.7 | dist: 1.4142
4. The Matrix Reloaded (2003)
   Жанры: Action,  Sci-Fi
   ★ 7.2 | dist: 0.2478
5. Arlingto